# BigMart Sales 3  (Analytics Vidhya)
## Rank 155 >> RMSE = 1146.7149

In [ ]:
import pandas as pd
import numpy as np

In [2]:
train_df = pd.read_csv('train.txt')
test_df = pd.read_csv('test.txt')
print('shape: ', train_df.shape)
train_df.head()

shape:  (8523, 12)


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [3]:
train_df["Data"] = "train"
test_df["Item_Outlet_Sales"] = np.nan
test_df["Data"] = "test"

In [4]:
df = pd.concat([train_df, test_df], axis=0)
print(df.shape)

(14204, 13)


In [5]:
del df["Outlet_Size"]
del df["Item_Identifier"]
del df["Outlet_Identifier"]

In [6]:
df.isnull().sum()

Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Establishment_Year       0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
Data                            0
dtype: int64

In [7]:
median = df["Item_Weight"].median()
df["Item_Weight"].fillna(median, inplace=True)

In [8]:
col = df.select_dtypes('object').columns
col = col[:-1]
df = pd.get_dummies(df, columns=col, drop_first=True)
df.head()

,Item_Weight,Item_Visibility,Item_MRP,Outlet_Establishment_Year,Item_Outlet_Sales,Data,Item_Fat_Content_Low Fat,Item_Fat_Content_Regular,Item_Fat_Content_low fat,Item_Fat_Content_reg,...,Item_Type_Others,Item_Type_Seafood,Item_Type_Snack Foods,Item_Type_Soft Drinks,Item_Type_Starchy Foods,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.30,0.016047,249.8092,1999,3735.1380,train,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,5.92,0.019278,48.2692,2009,443.4228,train,0,1,0,0,...,0,0,0,1,0,0,1,0,1,0
2,17.50,0.016760,141.6180,1999,2097.2700,train,1,0,0,0,...,0,0,0,0,0,0,0,1,0,0
3,19.20,0.000000,182.0950,1998,732.3800,train,0,1,0,0,...,0,0,0,0,0,0,1,0,0,0
4,8.93,0.000000,53.8614,1987,994.7052,train,1,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [9]:
train_df = df.loc[df['Data']=="train"]
test_df = df.loc[df['Data'] == 'test']
del train_df["Data"]
test_df.drop(["Data","Item_Outlet_Sales"],axis = 1,inplace = True)

C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


# Model Building

In [10]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
from math import *
from sklearn.model_selection import train_test_split, RandomizedSearchCV

In [11]:
train_sub,test_sub = train_test_split(train_df,test_size = 0.2,random_state = 42)
train_sub.shape,test_sub.shape

((6818, 29), (1705, 29))

In [12]:
x_train = train_sub.drop('Item_Outlet_Sales',axis = 1)
y_train = train_sub['Item_Outlet_Sales']
x_test = test_sub.drop('Item_Outlet_Sales', axis=1)
y_test = test_sub['Item_Outlet_Sales']

In [13]:
X = train_df.drop(['Item_Outlet_Sales'], axis=1)
Y = train_df['Item_Outlet_Sales']

# CatBoost

In [14]:
from catboost import CatBoostRegressor

In [15]:
model = CatBoostRegressor()

In [16]:
params = {
#            'depth'         : [4,5,6,7],
          'learning_rate' : [0.008],
#           'iterations'    : [100]
          'loss_function': ['RMSE']
    }

In [17]:
random = RandomizedSearchCV(model, param_distributions=params, cv=5, n_iter=10)

In [18]:
random.fit(x_train, y_train)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


0:	learn: 1718.6257688	total: 171ms	remaining: 2m 50s
1:	learn: 1711.4083264	total: 209ms	remaining: 1m 44s
2:	learn: 1704.3018430	total: 238ms	remaining: 1m 19s
3:	learn: 1697.1153773	total: 266ms	remaining: 1m 6s
4:	learn: 1689.6534442	total: 295ms	remaining: 58.8s
5:	learn: 1682.6895280	total: 323ms	remaining: 53.6s
6:	learn: 1675.4693799	total: 352ms	remaining: 50s
7:	learn: 1668.6294170	total: 386ms	remaining: 47.8s
8:	learn: 1662.4946914	total: 416ms	remaining: 45.8s
9:	learn: 1655.4342806	total: 444ms	remaining: 44s
10:	learn: 1648.9877778	total: 473ms	remaining: 42.5s
11:	learn: 1642.4117200	total: 501ms	remaining: 41.3s
12:	learn: 1636.0881959	total: 530ms	remaining: 40.2s
13:	learn: 1629.7459063	total: 562ms	remaining: 39.6s
14:	learn: 1623.2729549	total: 597ms	remaining: 39.2s
15:	learn: 1617.0973134	total: 627ms	remaining: 38.6s
16:	learn: 1610.9685019	total: 655ms	remaining: 37.9s
17:	learn: 1604.9822400	total: 684ms	remaining: 37.3s
18:	learn: 1598.9012134	total: 712ms	re

153:	learn: 1182.8787153	total: 5.37s	remaining: 29.5s
154:	learn: 1181.5396407	total: 5.42s	remaining: 29.6s
155:	learn: 1180.1327723	total: 5.48s	remaining: 29.6s
156:	learn: 1178.9133634	total: 5.53s	remaining: 29.7s
157:	learn: 1177.8600657	total: 5.6s	remaining: 29.8s
158:	learn: 1176.9412289	total: 5.65s	remaining: 29.9s
159:	learn: 1175.6197801	total: 5.69s	remaining: 29.9s
160:	learn: 1174.2962831	total: 5.76s	remaining: 30s
161:	learn: 1173.0545810	total: 5.82s	remaining: 30.1s
162:	learn: 1171.8769944	total: 5.86s	remaining: 30.1s
163:	learn: 1170.7073092	total: 5.89s	remaining: 30s
164:	learn: 1169.5300882	total: 5.92s	remaining: 30s
165:	learn: 1168.4911951	total: 5.95s	remaining: 29.9s
166:	learn: 1167.3786897	total: 5.99s	remaining: 29.9s
167:	learn: 1166.1929867	total: 6.02s	remaining: 29.8s
168:	learn: 1165.1055534	total: 6.05s	remaining: 29.7s
169:	learn: 1164.0795563	total: 6.08s	remaining: 29.7s
170:	learn: 1163.3164468	total: 6.11s	remaining: 29.6s
171:	learn: 1162.

304:	learn: 1096.5768278	total: 10.3s	remaining: 23.6s
305:	learn: 1096.3618672	total: 10.4s	remaining: 23.5s
306:	learn: 1096.1181689	total: 10.4s	remaining: 23.5s
307:	learn: 1095.8515278	total: 10.4s	remaining: 23.5s
308:	learn: 1095.6976098	total: 10.5s	remaining: 23.4s
309:	learn: 1095.5389989	total: 10.5s	remaining: 23.4s
310:	learn: 1095.2735517	total: 10.5s	remaining: 23.3s
311:	learn: 1095.0382747	total: 10.6s	remaining: 23.3s
312:	learn: 1094.8611633	total: 10.6s	remaining: 23.2s
313:	learn: 1094.6930005	total: 10.6s	remaining: 23.2s
314:	learn: 1094.4601618	total: 10.6s	remaining: 23.2s
315:	learn: 1094.2450289	total: 10.7s	remaining: 23.1s
316:	learn: 1094.0754207	total: 10.7s	remaining: 23.1s
317:	learn: 1093.9364711	total: 10.7s	remaining: 23.1s
318:	learn: 1093.7511276	total: 10.8s	remaining: 23s
319:	learn: 1093.5979410	total: 10.9s	remaining: 23.1s
320:	learn: 1093.3719051	total: 10.9s	remaining: 23.1s
321:	learn: 1093.1828811	total: 10.9s	remaining: 23.1s
322:	learn: 

455:	learn: 1076.1341500	total: 15.8s	remaining: 18.9s
456:	learn: 1076.0548687	total: 15.9s	remaining: 18.9s
457:	learn: 1075.9723827	total: 15.9s	remaining: 18.8s
458:	learn: 1075.8341332	total: 15.9s	remaining: 18.8s
459:	learn: 1075.7663854	total: 16s	remaining: 18.8s
460:	learn: 1075.6778987	total: 16s	remaining: 18.7s
461:	learn: 1075.6304765	total: 16s	remaining: 18.7s
462:	learn: 1075.5623231	total: 16.1s	remaining: 18.6s
463:	learn: 1075.4943232	total: 16.1s	remaining: 18.6s
464:	learn: 1075.4043548	total: 16.1s	remaining: 18.6s
465:	learn: 1075.2706877	total: 16.2s	remaining: 18.5s
466:	learn: 1075.1695360	total: 16.2s	remaining: 18.5s
467:	learn: 1075.0198100	total: 16.2s	remaining: 18.4s
468:	learn: 1074.9584099	total: 16.2s	remaining: 18.4s
469:	learn: 1074.8578835	total: 16.3s	remaining: 18.4s
470:	learn: 1074.7937695	total: 16.3s	remaining: 18.3s
471:	learn: 1074.7371143	total: 16.3s	remaining: 18.3s
472:	learn: 1074.6724601	total: 16.4s	remaining: 18.2s
473:	learn: 1074

...

508:	learn: 1075.4674609	total: 18.9s	remaining: 18.2s
509:	learn: 1075.4232843	total: 18.9s	remaining: 18.2s
510:	learn: 1075.3863402	total: 19s	remaining: 18.2s
511:	learn: 1075.3091365	total: 19s	remaining: 18.1s
512:	learn: 1075.1698358	total: 19s	remaining: 18.1s
513:	learn: 1075.1006891	total: 19.1s	remaining: 18s
514:	learn: 1075.0276234	total: 19.1s	remaining: 18s
515:	learn: 1074.9915793	total: 19.1s	remaining: 17.9s
516:	learn: 1074.9021651	total: 19.2s	remaining: 17.9s
517:	learn: 1074.8610063	total: 19.2s	remaining: 17.9s
518:	learn: 1074.8166400	total: 19.2s	remaining: 17.8s
519:	learn: 1074.7817170	total: 19.3s	remaining: 17.8s
520:	learn: 1074.7338921	total: 19.3s	remaining: 17.7s
521:	learn: 1074.6475091	total: 19.3s	remaining: 17.7s
522:	learn: 1074.6225672	total: 19.4s	remaining: 17.7s
523:	learn: 1074.5719917	total: 19.4s	remaining: 17.6s
524:	learn: 1074.5333362	total: 19.4s	remaining: 17.6s
525:	learn: 1074.4238764	total: 19.5s	remaining: 17.5s
526:	learn: 1074.392

659:	learn: 1067.5678675	total: 24.2s	remaining: 12.5s
660:	learn: 1067.5321287	total: 24.3s	remaining: 12.5s
661:	learn: 1067.4749735	total: 24.3s	remaining: 12.4s
662:	learn: 1067.4015979	total: 24.4s	remaining: 12.4s
663:	learn: 1067.3759452	total: 24.4s	remaining: 12.3s
664:	learn: 1067.3386771	total: 24.4s	remaining: 12.3s
665:	learn: 1067.2883037	total: 24.4s	remaining: 12.3s
666:	learn: 1067.2508609	total: 24.5s	remaining: 12.2s
667:	learn: 1067.2074682	total: 24.5s	remaining: 12.2s
668:	learn: 1067.1343833	total: 24.5s	remaining: 12.1s
669:	learn: 1067.0849924	total: 24.6s	remaining: 12.1s
670:	learn: 1067.0103165	total: 24.6s	remaining: 12.1s
671:	learn: 1066.9825576	total: 24.6s	remaining: 12s
672:	learn: 1066.9352929	total: 24.7s	remaining: 12s
673:	learn: 1066.8933298	total: 24.7s	remaining: 11.9s
674:	learn: 1066.8367391	total: 24.7s	remaining: 11.9s
675:	learn: 1066.7573847	total: 24.8s	remaining: 11.9s
676:	learn: 1066.6969551	total: 24.8s	remaining: 11.8s
677:	learn: 10

813:	learn: 1061.3864490	total: 29.3s	remaining: 6.69s
814:	learn: 1061.3570597	total: 29.3s	remaining: 6.65s
815:	learn: 1061.3395958	total: 29.3s	remaining: 6.62s
816:	learn: 1061.3216113	total: 29.4s	remaining: 6.58s
817:	learn: 1061.2805603	total: 29.4s	remaining: 6.54s
818:	learn: 1061.2313184	total: 29.4s	remaining: 6.5s
819:	learn: 1061.2080585	total: 29.5s	remaining: 6.47s
820:	learn: 1061.1450201	total: 29.5s	remaining: 6.43s
821:	learn: 1061.1169753	total: 29.5s	remaining: 6.39s
822:	learn: 1061.0819896	total: 29.6s	remaining: 6.36s
823:	learn: 1061.0584895	total: 29.6s	remaining: 6.32s
824:	learn: 1061.0491171	total: 29.6s	remaining: 6.28s
825:	learn: 1060.9933994	total: 29.7s	remaining: 6.25s
826:	learn: 1060.9479716	total: 29.7s	remaining: 6.21s
827:	learn: 1060.8956215	total: 29.7s	remaining: 6.17s
828:	learn: 1060.8628346	total: 29.8s	remaining: 6.14s
829:	learn: 1060.8358794	total: 29.8s	remaining: 6.1s
830:	learn: 1060.7892240	total: 29.8s	remaining: 6.06s
831:	learn: 

964:	learn: 1056.8742924	total: 34.4s	remaining: 1.25s
965:	learn: 1056.8088316	total: 34.4s	remaining: 1.21s
966:	learn: 1056.7938754	total: 34.4s	remaining: 1.18s
967:	learn: 1056.7756639	total: 34.5s	remaining: 1.14s
968:	learn: 1056.7432545	total: 34.5s	remaining: 1.1s
969:	learn: 1056.7225147	total: 34.5s	remaining: 1.07s
970:	learn: 1056.7200716	total: 34.6s	remaining: 1.03s
971:	learn: 1056.7054544	total: 34.6s	remaining: 996ms
972:	learn: 1056.6973773	total: 34.6s	remaining: 961ms
973:	learn: 1056.6791536	total: 34.6s	remaining: 925ms
974:	learn: 1056.6567035	total: 34.7s	remaining: 889ms
975:	learn: 1056.6368580	total: 34.7s	remaining: 854ms
976:	learn: 1056.5969459	total: 34.7s	remaining: 818ms
977:	learn: 1056.5764824	total: 34.8s	remaining: 782ms
978:	learn: 1056.5595937	total: 34.8s	remaining: 747ms
979:	learn: 1056.5533184	total: 34.8s	remaining: 711ms
980:	learn: 1056.5156997	total: 34.9s	remaining: 675ms
981:	learn: 1056.4917051	total: 34.9s	remaining: 640ms
982:	learn:

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=<catboost.core.CatBoostRegressor object at 0x000002B694B5B748>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': [0.008],
                                        'loss_function': ['RMSE']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [19]:
random.best_params_

{'loss_function': 'RMSE', 'learning_rate': 0.008}

In [20]:
def report(results, n_top=3):
    for i in range(1, n_top + 1):
        candidates = np.flatnonzero(results['rank_test_score'] == i)
        for candidate in candidates:
            print("Model with rank: {0}".format(i))
            print("Mean validation score: {0:.3f} (std: {1:.5f})".format(
                  results['mean_test_score'][candidate],
                  results['std_test_score'][candidate]))
            print("Parameters: {0}".format(results['params'][candidate]))
            print("")
report(random.cv_results_,5)

Model with rank: 1
Mean validation score: 0.594 (std: 0.01056)
Parameters: {'loss_function': 'RMSE', 'learning_rate': 0.008}



In [26]:
y_pred = random.predict(x_test)

In [27]:
rmse = sqrt(mean_squared_error(y_test, y_pred))
rmse

1020.8891761110615

In [28]:
r2_score(y_test, y_pred)

0.6165466563809325

# Test Dataset

In [29]:
random.fit(X,Y)

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\model_selection\_search.py:266: UserWarning: The total space of parameters 1 is smaller than n_iter=10. Running 1 iterations. For exhaustive searches, use GridSearchCV.
  % (grid_size, self.n_iter, grid_size), UserWarning)


0:	learn: 1686.0443011	total: 66.4ms	remaining: 1m 6s
1:	learn: 1678.9961346	total: 119ms	remaining: 59.2s
2:	learn: 1671.8939884	total: 178ms	remaining: 59.2s
3:	learn: 1665.3790887	total: 235ms	remaining: 58.5s
4:	learn: 1658.5242060	total: 301ms	remaining: 59.9s
5:	learn: 1651.7448795	total: 352ms	remaining: 58.4s
6:	learn: 1644.8234701	total: 404ms	remaining: 57.4s
7:	learn: 1638.0704366	total: 459ms	remaining: 56.9s
8:	learn: 1631.4498052	total: 529ms	remaining: 58.2s
9:	learn: 1624.8351287	total: 582ms	remaining: 57.7s
10:	learn: 1618.1702888	total: 640ms	remaining: 57.6s
11:	learn: 1611.6975713	total: 682ms	remaining: 56.2s
12:	learn: 1605.1565762	total: 732ms	remaining: 55.6s
13:	learn: 1598.8206020	total: 776ms	remaining: 54.7s
14:	learn: 1593.0360296	total: 802ms	remaining: 52.7s
15:	learn: 1586.8696739	total: 836ms	remaining: 51.4s
16:	learn: 1581.5545954	total: 868ms	remaining: 50.2s
17:	learn: 1575.1436795	total: 902ms	remaining: 49.2s
18:	learn: 1569.0531604	total: 957ms	

153:	learn: 1157.3872548	total: 5.63s	remaining: 30.9s
154:	learn: 1156.0771973	total: 5.69s	remaining: 31s
155:	learn: 1154.7940999	total: 5.76s	remaining: 31.2s
156:	learn: 1153.6508531	total: 5.82s	remaining: 31.2s
157:	learn: 1152.4485935	total: 5.87s	remaining: 31.3s
158:	learn: 1151.1846215	total: 5.9s	remaining: 31.2s
159:	learn: 1149.9769584	total: 5.94s	remaining: 31.2s
160:	learn: 1148.8923880	total: 5.97s	remaining: 31.1s
161:	learn: 1147.8831419	total: 6s	remaining: 31s
162:	learn: 1146.7778291	total: 6.04s	remaining: 31s
163:	learn: 1145.8088541	total: 6.07s	remaining: 31s
164:	learn: 1144.8543005	total: 6.11s	remaining: 30.9s
165:	learn: 1143.6695893	total: 6.14s	remaining: 30.8s
166:	learn: 1142.6601515	total: 6.17s	remaining: 30.8s
167:	learn: 1141.7853072	total: 6.2s	remaining: 30.7s
168:	learn: 1140.7095253	total: 6.24s	remaining: 30.7s
169:	learn: 1139.7445693	total: 6.27s	remaining: 30.6s
170:	learn: 1138.7048265	total: 6.3s	remaining: 30.6s
171:	learn: 1137.7769440

305:	learn: 1076.2466410	total: 11.4s	remaining: 25.9s
306:	learn: 1076.1047614	total: 11.5s	remaining: 25.9s
307:	learn: 1075.9280398	total: 11.5s	remaining: 25.8s
308:	learn: 1075.7258820	total: 11.5s	remaining: 25.8s
309:	learn: 1075.5545655	total: 11.6s	remaining: 25.7s
310:	learn: 1075.3833266	total: 11.6s	remaining: 25.7s
311:	learn: 1075.1870347	total: 11.6s	remaining: 25.6s
312:	learn: 1075.0422208	total: 11.7s	remaining: 25.6s
313:	learn: 1074.8183265	total: 11.7s	remaining: 25.5s
314:	learn: 1074.6782638	total: 11.7s	remaining: 25.5s
315:	learn: 1074.4934661	total: 11.7s	remaining: 25.4s
316:	learn: 1074.3223455	total: 11.8s	remaining: 25.4s
317:	learn: 1074.1304464	total: 11.8s	remaining: 25.3s
318:	learn: 1073.9917740	total: 11.8s	remaining: 25.3s
319:	learn: 1073.8330748	total: 11.9s	remaining: 25.2s
320:	learn: 1073.6191498	total: 11.9s	remaining: 25.2s
321:	learn: 1073.5224168	total: 11.9s	remaining: 25.2s
322:	learn: 1073.3112189	total: 12s	remaining: 25.1s
323:	learn: 

455:	learn: 1059.2117618	total: 16.9s	remaining: 20.2s
456:	learn: 1059.1075415	total: 17s	remaining: 20.2s
457:	learn: 1059.0475009	total: 17s	remaining: 20.1s
458:	learn: 1058.9682243	total: 17s	remaining: 20.1s
459:	learn: 1058.9273101	total: 17.1s	remaining: 20s
460:	learn: 1058.8597270	total: 17.1s	remaining: 20s
461:	learn: 1058.8046663	total: 17.1s	remaining: 19.9s
462:	learn: 1058.7214180	total: 17.2s	remaining: 19.9s
463:	learn: 1058.6378429	total: 17.2s	remaining: 19.9s
464:	learn: 1058.5612235	total: 17.2s	remaining: 19.8s
465:	learn: 1058.5092116	total: 17.3s	remaining: 19.8s
466:	learn: 1058.4466690	total: 17.3s	remaining: 19.7s
467:	learn: 1058.3860984	total: 17.3s	remaining: 19.7s
468:	learn: 1058.3041653	total: 17.4s	remaining: 19.7s
469:	learn: 1058.2252621	total: 17.4s	remaining: 19.6s
470:	learn: 1058.1707524	total: 17.4s	remaining: 19.6s
471:	learn: 1058.0804720	total: 17.5s	remaining: 19.5s
472:	learn: 1058.0188282	total: 17.5s	remaining: 19.5s
473:	learn: 1057.945

...

505:	learn: 1062.7421352	total: 20.2s	remaining: 19.7s
506:	learn: 1062.6803497	total: 20.2s	remaining: 19.7s
507:	learn: 1062.5967226	total: 20.3s	remaining: 19.7s
508:	learn: 1062.5308375	total: 20.3s	remaining: 19.6s
509:	learn: 1062.4429812	total: 20.4s	remaining: 19.6s
510:	learn: 1062.4133368	total: 20.5s	remaining: 19.6s
511:	learn: 1062.3585902	total: 20.5s	remaining: 19.6s
512:	learn: 1062.2998016	total: 20.6s	remaining: 19.5s
513:	learn: 1062.2607128	total: 20.6s	remaining: 19.5s
514:	learn: 1062.1999230	total: 20.6s	remaining: 19.4s
515:	learn: 1062.1503142	total: 20.7s	remaining: 19.4s
516:	learn: 1062.0725227	total: 20.7s	remaining: 19.3s
517:	learn: 1062.0284036	total: 20.7s	remaining: 19.3s
518:	learn: 1061.9857635	total: 20.8s	remaining: 19.3s
519:	learn: 1061.9491919	total: 20.8s	remaining: 19.2s
520:	learn: 1061.8826846	total: 20.9s	remaining: 19.2s
521:	learn: 1061.7958658	total: 20.9s	remaining: 19.1s
522:	learn: 1061.7410419	total: 20.9s	remaining: 19.1s
523:	learn

655:	learn: 1055.6080421	total: 25.3s	remaining: 13.3s
656:	learn: 1055.5571019	total: 25.3s	remaining: 13.2s
657:	learn: 1055.5330035	total: 25.4s	remaining: 13.2s
658:	learn: 1055.5077211	total: 25.5s	remaining: 13.2s
659:	learn: 1055.4798238	total: 25.6s	remaining: 13.2s
660:	learn: 1055.4436980	total: 25.6s	remaining: 13.1s
661:	learn: 1055.3954546	total: 25.7s	remaining: 13.1s
662:	learn: 1055.3746917	total: 25.7s	remaining: 13.1s
663:	learn: 1055.3422770	total: 25.8s	remaining: 13s
664:	learn: 1055.3184463	total: 25.8s	remaining: 13s
665:	learn: 1055.2865053	total: 25.8s	remaining: 13s
666:	learn: 1055.2318561	total: 25.9s	remaining: 12.9s
667:	learn: 1055.1867620	total: 25.9s	remaining: 12.9s
668:	learn: 1055.1565510	total: 25.9s	remaining: 12.8s
669:	learn: 1055.1168134	total: 26s	remaining: 12.8s
670:	learn: 1055.0736788	total: 26s	remaining: 12.7s
671:	learn: 1055.0334456	total: 26s	remaining: 12.7s
672:	learn: 1055.0009538	total: 26.1s	remaining: 12.7s
673:	learn: 1054.95057

807:	learn: 1050.4121665	total: 31.1s	remaining: 7.39s
808:	learn: 1050.3805591	total: 31.1s	remaining: 7.35s
809:	learn: 1050.3618458	total: 31.2s	remaining: 7.31s
810:	learn: 1050.3472767	total: 31.2s	remaining: 7.27s
811:	learn: 1050.3226729	total: 31.2s	remaining: 7.24s
812:	learn: 1050.2615622	total: 31.3s	remaining: 7.2s
813:	learn: 1050.2284722	total: 31.3s	remaining: 7.16s
814:	learn: 1050.1865990	total: 31.4s	remaining: 7.12s
815:	learn: 1050.1590021	total: 31.4s	remaining: 7.08s
816:	learn: 1050.1379333	total: 31.4s	remaining: 7.04s
817:	learn: 1050.1071314	total: 31.4s	remaining: 7s
818:	learn: 1050.0749011	total: 31.5s	remaining: 6.96s
819:	learn: 1050.0455987	total: 31.5s	remaining: 6.92s
820:	learn: 1050.0142883	total: 31.6s	remaining: 6.88s
821:	learn: 1049.9937576	total: 31.6s	remaining: 6.84s
822:	learn: 1049.9732098	total: 31.6s	remaining: 6.8s
823:	learn: 1049.9568969	total: 31.7s	remaining: 6.76s
824:	learn: 1049.9256895	total: 31.7s	remaining: 6.72s
825:	learn: 104

960:	learn: 1046.6758449	total: 36.5s	remaining: 1.48s
961:	learn: 1046.6552996	total: 36.6s	remaining: 1.44s
962:	learn: 1046.6412121	total: 36.6s	remaining: 1.41s
963:	learn: 1046.6179756	total: 36.6s	remaining: 1.37s
964:	learn: 1046.6000680	total: 36.7s	remaining: 1.33s
965:	learn: 1046.5826538	total: 36.7s	remaining: 1.29s
966:	learn: 1046.5707554	total: 36.7s	remaining: 1.25s
967:	learn: 1046.5372307	total: 36.8s	remaining: 1.22s
968:	learn: 1046.5253086	total: 36.8s	remaining: 1.18s
969:	learn: 1046.5100469	total: 36.8s	remaining: 1.14s
970:	learn: 1046.4878712	total: 36.9s	remaining: 1.1s
971:	learn: 1046.4727726	total: 36.9s	remaining: 1.06s
972:	learn: 1046.4500229	total: 36.9s	remaining: 1.02s
973:	learn: 1046.4326585	total: 37s	remaining: 987ms
974:	learn: 1046.4075250	total: 37s	remaining: 949ms
975:	learn: 1046.3761735	total: 37.1s	remaining: 911ms
976:	learn: 1046.3530135	total: 37.1s	remaining: 874ms
977:	learn: 1046.3277276	total: 37.2s	remaining: 836ms
978:	learn: 104

RandomizedSearchCV(cv=5, error_score='raise-deprecating',
                   estimator=<catboost.core.CatBoostRegressor object at 0x000002B694B5B748>,
                   iid='warn', n_iter=10, n_jobs=None,
                   param_distributions={'learning_rate': [0.008],
                                        'loss_function': ['RMSE']},
                   pre_dispatch='2*n_jobs', random_state=None, refit=True,
                   return_train_score=False, scoring=None, verbose=0)

In [30]:
test_df_pred = random.predict(test_df)

In [32]:
submission = pd.read_csv('SampleSubmission.csv')
submission['Item_Outlet_Sales'] = test_df_pred
submission.head()

,Item_Identifier,Outlet_Identifier,Item_Outlet_Sales
0,FDW58,OUT049,1741.646818
1,FDW14,OUT017,1400.317631
2,NCN55,OUT010,891.893230
3,FDQ58,OUT017,2544.280915
4,FDY38,OUT027,5796.639711


In [33]:
submission.to_csv('SampleSubmission.csv', index=False)  # RMSE = 1146.7149 --> Rank 155 